Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [14]:
L2_coeff = 0.5e-2
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    X_test = tf.constant(test_dataset)
    
    y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b1 = tf.Variable(tf.zeros([1024]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # in session 
    # tf.initialize_all_variables().run()
    
    logits = tf.matmul(X,W1) + b1
    logits_test = tf.matmul(X_test,W1) + b1
    
    layer1 = tf.nn.relu(logits)
    layer1_test = tf.nn.relu(logits_test)
    
    out = tf.matmul(layer1, W2) + b2
    out_test = tf.matmul(layer1_test, W2) + b2
    
    train_prediction = tf.nn.softmax(out)

    nn        = tf.nn.softmax_cross_entropy_with_logits(out, y)
    loss      = tf.reduce_mean(nn) + L2_coeff * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)) # <--
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    test_prediction = tf.nn.softmax(out_test)

In [16]:
batch_size = 128
with tf.Session(graph=graph) as sess:
    
    tf.initialize_all_variables().run()
    
    for i in range(3000):
        idx = np.random.randint(train_dataset.shape[0], size=batch_size)
        feed_dict = {X : train_dataset[idx], y : train_labels[idx]}
        _, proba = sess.run([optimizer, train_prediction], feed_dict=feed_dict)
        if (i%500 == 0):
            print("%d - Test accuracy: %.1f%%" % (i, accuracy(test_prediction.eval(), test_labels)))

    print("\nTest accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 - Test accuracy: 35.6%
500 - Test accuracy: 89.1%
1000 - Test accuracy: 91.4%
1500 - Test accuracy: 91.7%
2000 - Test accuracy: 91.7%
2500 - Test accuracy: 91.5%

Test accuracy: 91.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
batch_size = 128
with tf.Session(graph=graph) as sess:
    
    tf.initialize_all_variables().run()
    
    for i in range(3000):
        idx = np.random.randint(train_dataset.shape[0]/1000, size=batch_size) # <-- less data /1000
        feed_dict = {X : train_dataset[idx], y : train_labels[idx]}
        _, proba = sess.run([optimizer, train_prediction], feed_dict=feed_dict)
        if (i%500 == 0):
            print("%d - Test accuracy: %.1f%%" % (i, accuracy(test_prediction.eval(), test_labels)))

    print("\nTest accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 - Test accuracy: 31.3%
500 - Test accuracy: 76.5%
1000 - Test accuracy: 78.4%
1500 - Test accuracy: 80.1%
2000 - Test accuracy: 79.8%
2500 - Test accuracy: 79.8%

Test accuracy: 79.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [19]:
L2_coeff = 0.5e-2
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    X_test = tf.constant(test_dataset)
    
    y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b1 = tf.Variable(tf.zeros([1024]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # in session 
    # tf.initialize_all_variables().run()
    
    logits = tf.matmul(X,W1) + b1
    logits_test = tf.matmul(X_test,W1) + b1
    
    layer1 = tf.nn.relu(logits)
    layer1 = tf.nn.dropout(layer1, 0.5) # <-- dropout
    layer1_test = tf.nn.relu(logits_test)
    
    out = tf.matmul(layer1, W2) + b2
    out_test = tf.matmul(layer1_test, W2) + b2
    
    train_prediction = tf.nn.softmax(out)

    nn        = tf.nn.softmax_cross_entropy_with_logits(out, y)
    loss      = tf.reduce_mean(nn) + L2_coeff * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)) # <--
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    test_prediction = tf.nn.softmax(out_test)

In [20]:
batch_size = 128
with tf.Session(graph=graph) as sess:
    
    tf.initialize_all_variables().run()
    
    for i in range(3000):
        idx = np.random.randint(train_dataset.shape[0], size=batch_size)
        feed_dict = {X : train_dataset[idx], y : train_labels[idx]}
        _, proba = sess.run([optimizer, train_prediction], feed_dict=feed_dict)
        if (i%500 == 0):
            print("%d - Test accuracy: %.1f%%" % (i, accuracy(test_prediction.eval(), test_labels)))

    print("\nTest accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 - Test accuracy: 27.6%
500 - Test accuracy: 88.8%
1000 - Test accuracy: 91.3%
1500 - Test accuracy: 90.7%
2000 - Test accuracy: 90.6%
2500 - Test accuracy: 91.4%

Test accuracy: 91.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [46]:
L2_coeff = 0.5e-2
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    X_test = tf.constant(test_dataset)
    
    y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    W2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b1 = tf.Variable(tf.zeros([1024]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # in session 
    # tf.initialize_all_variables().run()
    
    logits = tf.matmul(X,W1) + b1
    logits_test = tf.matmul(X_test,W1) + b1
    
    layer1 = tf.nn.relu(logits)
    layer1 = tf.nn.dropout(layer1, 0.5) # <-- dropout
    layer1_test = tf.nn.relu(logits_test)
    
    out = tf.matmul(layer1, W2) + b2
    out_test = tf.matmul(layer1_test, W2) + b2
    
    train_prediction = tf.nn.softmax(out)

    nn        = tf.nn.softmax_cross_entropy_with_logits(out, y)
    loss      = tf.reduce_mean(nn) + L2_coeff * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)) # <-- l2
    
    global_step = tf.Variable(0, trainable=False)
    learn_rate  = tf.train.exponential_decay(0.5, global_step, 1000, 0.8) # <-- learning rate
    #learn_rate = tf.constant(0.5)
    optimizer   = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss, global_step=global_step)
    
    test_prediction = tf.nn.softmax(out_test)

In [47]:
batch_size = 128
with tf.Session(graph=graph) as sess:
    
    tf.initialize_all_variables().run()
    
    for i in range(5000):
        idx = np.random.randint(train_dataset.shape[0], size=batch_size)
        feed_dict = {X : train_dataset[idx], y : train_labels[idx]}
        _, proba = sess.run([optimizer, train_prediction], feed_dict=feed_dict)
        if (i%500 == 0):
            print("Test accuracy: %.1f%%, learn rate: %.2f, step %d" % (accuracy(test_prediction.eval(), test_labels), learn_rate.eval(), global_step.eval()))

    print("\nTest accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Test accuracy: 46.6%, learn rate: 0.50, step 1
Test accuracy: 88.9%, learn rate: 0.45, step 501
Test accuracy: 90.7%, learn rate: 0.40, step 1001
Test accuracy: 91.0%, learn rate: 0.36, step 1501
Test accuracy: 91.3%, learn rate: 0.32, step 2001
Test accuracy: 91.6%, learn rate: 0.29, step 2501
Test accuracy: 91.9%, learn rate: 0.26, step 3001
Test accuracy: 91.7%, learn rate: 0.23, step 3501
Test accuracy: 92.1%, learn rate: 0.20, step 4001
Test accuracy: 91.7%, learn rate: 0.18, step 4501

Test accuracy: 91.9%
